In [ ]:
!pip install numpy pandas scikit-learn


In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
#from google.colab import files  # This import is not used, so we can remove it.

# Load the data first
df_movies = pd.read_csv("/content/movies.csv")
df_ratings = pd.read_csv("/content/ratings.csv")

# Now you can create the user-item matrix
user_item_matrix = df_ratings.pivot(index='userId', columns='movieId', values='rating')
user_item_matrix.fillna(0, inplace=True)

In [ ]:
sparse_matrix = csr_matrix(user_item_matrix)


In [ ]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5)
model_knn.fit(sparse_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [ ]:
def recommend_movies(user_id, num_recommendations=5):
    if user_id not in user_item_matrix.index:
        return "User not found!"

    user_index = user_item_matrix.index.get_loc(user_id)
    distances, indices = model_knn.kneighbors(user_item_matrix.iloc[user_index, :].values.reshape(1, -1), n_neighbors=6)

    similar_users = indices.flatten()[1:]  # Exclude the user themselves
    recommended_movies = df_ratings[df_ratings['userId'].isin(similar_users)].groupby('movieId')['rating'].mean()
    recommended_movies = recommended_movies.sort_values(ascending=False).head(num_recommendations)

    return df_movies[df_movies['movieId'].isin(recommended_movies.index)][['title']]


In [ ]:
user_id = 4000  # Change to a valid user ID
print(recommend_movies(user_id))


                                 title
314   Shawshank Redemption, The (1994)
582          Dances with Wolves (1990)
784                   Lone Star (1996)
1013                   Die Hard (1988)
1182                 Goodfellas (1990)
